In [2]:
import session_info
session_info.show()

/opt/conda/lib/python3.8/site-packages/jupyter_server/transutils.py:13: FutureWarning: The alias `_()` will be deprecated. Use `_i18n()` instead.
  warnings.warn(warn_msg, FutureWarning)


In [1]:
from fpdf import FPDF
import fpdf
import pandas as pd
pd.options.plotting.backend = "plotly"
import plotly as plt
import pickle
from sentence_transformers import SentenceTransformer, util
import numpy as np
import plotly.express as px

In [7]:
k31_full = pickle.load(open('../Model_Selection/k_31_full', 'rb'))
cluster_label_bigrams = pickle.load(open('cluster_label_bigrams','rb')) 
cluster_importance = pickle.load(open('cluster_importance', 'rb'))

/opt/conda/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator KMeans from version 0.23.1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [14]:
def get_plot(cv): #takes in list of skills and returns a plot with score for each cluster
    df_sim = pd.DataFrame()
    df_sim['cluster'] = range(len(k31_full.cluster_centers_))
    
    labels = cluster_label_bigrams
    importance = cluster_importance
    
    model = 'all-distilroberta-v1'
    model = SentenceTransformer(model)
    embeddings_cv = model.encode(cv)
    embeddings_f = embeddings_cv.astype(float)
    clusters_cv = k31_full.predict(embeddings_f)
    clusters_cv_l  = clusters_cv.tolist()
    
    cv_scores = []
    for i, cluster in enumerate(clusters_cv):
        cv_scores.append(util.pytorch_cos_sim(k31_full.cluster_centers_[cluster], embeddings_f[i]).item())
    
    scores = []
    for cluster in range(len(k31_full.cluster_centers_)):
        if cluster not in clusters_cv_l:
            scores.append(0)
        else:
            score = 0
            indexes = np.where(clusters_cv==cluster)[0]
            for i in indexes:
                if cv_scores[i] > score:
                    score = cv_scores[i]
            scores.append(score)   
    
    df_sim['score'] = scores
    df_sim['importance'] = importance
    df_sim['labels'] = labels
    
    df_sim['CV_similarity'] = df_sim['importance']*df_sim['score']
    df_sim['Importance_Cluster_in_Job_Postings'] = df_sim['importance']-df_sim['CV_similarity']
    df_sim = df_sim.sort_values('importance')
    
    fig = px.bar(df_sim, y='labels', x=["CV_similarity","Importance_Cluster_in_Job_Postings"], hover_data = ['importance'])
    fig.update_layout(height=800, \
                          title = 'Author\'s CV similarity to requirement clusters in context of relative cluster presence',\
                          barmode='stack', \
                          yaxis_title="3 most common Bigrams in Cluster",\
                          xaxis_title="CV Similarity to Cluster")
    fig.show()
    return fig
    
plot = get_plot(['i like python','i am a great communicator'])

In [16]:
plot.write_image("fig1.png")

In [58]:
pdf=FPDF()
pdf.add_page()
#pltx="fig1.png"
pdf.image('fig1.png',x = 10, y = 10, w = 100, h = 100, type = '', link = '')
pdf.set_font('Arial', 'B', 16)
pdf.cell(40, 0, 'Hello World! this string asdfasdf')
pdf.set_font('Arial', '', 12)
pdf.cell(20,250, 'And here some explanation but what happens if we add more text', align = "L", fill = 'T', border='1')

In [59]:
pdf.output('tuto2.pdf', 'F')

''